In [40]:
import math
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import os

import shutil
import sklearn
from sklearn.model_selection import KFold
import gpytorch
from gpytorch.models import ExactGP
from gpytorch.likelihoods import DirichletClassificationLikelihood
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, RBFKernel

from sklearn.metrics import confusion_matrix
import itertools
from sklearn.metrics import precision_score, recall_score, roc_auc_score, matthews_corrcoef, balanced_accuracy_score, confusion_matrix, f1_score, roc_curve,precision_recall_curve, auc

import sys
sys.path.append('../../')
from RF_GSCV import * # RF_GSCV contains the calculate metrics function to get the TP, TN, FP, FN scores 


In [41]:

class DirichletGPModel(ExactGP):
    """
    A Dirichlet Gaussian Process (GP) model for multi-class classification.

    This model uses a Gaussian Process with a Dirichlet prior to handle multi-class classification tasks.
    It extends the ExactGP class from GPyTorch, a library for Gaussian Processes in PyTorch.

    Attributes:
        mean_module (gpytorch.means.ConstantMean): The mean module for the GP, initialized with a constant mean function for each class.
        covar_module (gpytorch.kernels.ScaleKernel): The covariance module for the GP, using a scaled RBF kernel for each class.

    Args:
        train_x (torch.Tensor): Training data features.
        train_y (torch.Tensor): Training data labels.
        likelihood (gpytorch.likelihoods.Likelihood): The likelihood function.
        num_classes (int): The number of classes for the classification task.
    """
    def __init__(self, train_x, train_y, likelihood, num_classes):
        super(DirichletGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = ConstantMean(batch_shape=torch.Size((num_classes,)))
        self.covar_module = ScaleKernel(
            RBFKernel(batch_shape=torch.Size((num_classes,))),
            batch_shape=torch.Size((num_classes,)),
        )

    def forward(self, x):
        """
        Forward pass through the GP model.

        Args:
            x (torch.Tensor): Input data features.

        Returns:
            gpytorch.distributions.MultivariateNormal: The multivariate normal distribution representing the GP posterior.
        """
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


In [42]:
class Trainer: 
    def __init__(self,model, likelihood, iterations): 
        self.model = model
        self.likelihood = likelihood 
        smoke_test = ('CI' in os.environ)
        self.n_iterations = 2 if smoke_test else iterations
        self.optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
        self.loss_fn = gpytorch.mlls.ExactMarginalLogLikelihood(self.likelihood, self.model)
        
    def train(self, train_x, train_y): 
        self.model.train()
        self.likelihood.train()
        predictions = [] 
        for i in range(self.n_iterations): 
            self.optimizer.zero_grad()
            output = self.model(train_x)
            loss = -self.loss_fn(output, self.likelihood.transformed_targets).sum()
            loss.backward()
            if (i%10==0): 
                print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
                    i + 1, self.n_iterations, loss.item(),
                    self.model.covar_module.base_kernel.lengthscale.mean().item(),
                    self.model.likelihood.second_noise_covar.noise.mean().item()
                ))
             
          
            self.optimizer.step() 


    def predict(self, input): 
        """
        Make predictions using the GP model.

        Args:
            input (torch.Tensor): The input data for making predictions.
        
        Returns:
            dist (gpytorch.distributions.MultivariateNormal): The distribution representing the GP posterior.
            observed_pred (gpytorch.distributions.MultivariateNormal): The predicted distribution considering the likelihood.
            pred_means (torch.Tensor): The means of the predicted distributions.
            class_pred (torch.Tensor): The predicted class labels.
        """
        self.model.eval()
        self.likelihood.eval()

        with gpytorch.settings.fast_pred_var(), torch.no_grad():
            dist = self.model(input)     # output distribution
            pred_means = dist.loc          # means of distributino 
            observed_pred = self.likelihood(self.model(input))    # likelihood predictions mean and var  

            class_pred = self.model(input).loc.max(0)[1]
            
        return dist, observed_pred, pred_means, class_pred
    

    def evaluate(self, x_input, y_true): 
        """
        Evaluate the GP model.

        Args:
            x_input (torch.Tensor): The input data features.
            y_true (torch.Tensor): The true labels for the input data.
        
        Returns:
            y_pred (numpy.ndarray): The predicted class labels.
        """
        y_pred = self.model(x_input).loc.max(0)[1].numpy()
        
        return y_pred

    def gp_results(self, x_input, y_true, plot_title=None): 
        """
        Calculate evaluation metrics and print results.

        Args:
            x_input (torch.Tensor): The input data features.
            y_true (torch.Tensor or numpy.ndarray): The true labels for the input data.
            plot_title (str, optional): The title for the confusion matrix plot.
        
        Returns:
            dict: A dictionary containing evaluation metrics and confusion matrix components.
        """
        y_pred = self.evaluate(x_input, y_true) 
        if isinstance(y_true, torch.Tensor):
            y_true = y_true.numpy().reshape(-1)
        # plot_confusion_matrix(y_true, y_pred, ['0','1'], title=plot_title)
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        dist = self.model(x_input)     # get predicted distributions 
        pred_means = dist.loc          # means for predicted dist  

        recall = recall_score(y_true, y_pred)
        tp, tn, fp, fn = calculate_metrics(y_true, y_pred) 
        sensitivity = tp / (tp + fn) 
        specificity = tn / (tn + fp) 
        cm = confusion_matrix(y_true, y_pred)
        print(f'accuracy: {accuracy:.4f}, precision: {precision:.4f}, recall: {recall:.4f}, specificity: {specificity:.4f}, cm: {cm}')
        return {'accuracy': accuracy, 'precision': precision,  'recall':recall, 'specificity':specificity, 'TN': tn, 'FN': fn, 'FP': fp, 'TP': tp }

       

In [4]:
def make_torch_tens(filepath, filename): 
    trainX_df = pd.read_csv(filepath+filename+'_trainX.csv')
    trainy_df = pd.read_csv(filepath+filename+'_train_y.csv')
    testX_df = pd.read_csv(filepath+filename+'_testX.csv')
    testy_df = pd.read_csv(filepath+filename+'_test_y.csv')

    train_x_temp = trainX_df.to_numpy().astype("long")
    test_x_temp = testX_df.to_numpy().astype("long")
    
    train_y_temp = trainy_df.to_numpy().flatten().astype("long")
    test_y_temp = testy_df.to_numpy().flatten().astype("long")
    
    trainX = torch.from_numpy(train_x_temp)
    trainy = torch.from_numpy(train_y_temp)
    testX = torch.from_numpy(test_x_temp)
    testy = torch.from_numpy(test_y_temp)
    print(f'train X: {trainX.shape}, train y: {trainy.shape}, test X: {testX.shape}, test y: {testy.shape}')
    return trainX, trainy, testX, testy
    


In [44]:
def save_results(trainX, trainy, testX, testy, root_name, n_iterations=300, n_samples=100):
    """
    Train a Dirichlet Gaussian Process model and save the training and test performance results.

    This function trains a Dirichlet GP model on the given training data, evaluates it on both the training
    and test data, and saves various performance metrics and predictions to pandas DataFrames.

    Args:
        trainX (torch.Tensor): The training data features.
        trainy (torch.Tensor): The training data labels.
        testX (torch.Tensor): The test data features.
        testy (torch.Tensor): The test data labels.
        root_name (str): The root name used for labeling the model in the results.
        n_iterations (int, optional): The number of training iterations. Default is 300.
        n_samples (int, optional): The number of samples for prediction. Default is 100.

    Returns:
        train_perf_df (pd.DataFrame): DataFrame containing performance metrics and predictions for the training data.
        test_perf_df (pd.DataFrame): DataFrame containing performance metrics and predictions for the test data.
    """
    likelihood = DirichletClassificationLikelihood(trainy, learn_additional_noise=True)
    model = DirichletGPModel(trainX, likelihood.transformed_targets, likelihood, num_classes=likelihood.num_classes)
    # n_iterations = 300
    trainer = Trainer(model, likelihood, n_iterations)
    trainer.train(trainX, trainy) 
  
    train_dist, train_observed_pred, train_pred_means, train_pred  = trainer.predict(trainX)
    train_results = trainer.gp_results(trainX, trainy)
    test_dist, test_observed_pred, test_pred_means, test_pred  = trainer.predict(testX)
    test_results = trainer.gp_results(testX, testy)
    
    train_observed_pred.mean.numpy()
    train_pred_variance2D = train_observed_pred.variance.numpy()
    test_observed_pred.mean.numpy()
    test_pred_variance2D=test_observed_pred.variance.numpy()
    
    train_pred_samples = train_dist.sample(torch.Size((256,))).exp()
    train_probabilities = (train_pred_samples / train_pred_samples.sum(-2, keepdim=True)).mean(0)

    train_prob_stds = (train_pred_samples / train_pred_samples.sum(-2, keepdim=True)).std(0)

    

    test_pred_samples = test_dist.sample(torch.Size((100,))).exp()

    test_probabilities = (test_pred_samples / test_pred_samples.sum(-2, keepdim=True)).mean(0)
    test_prob_stds = (test_pred_samples / test_pred_samples.sum(-2, keepdim=True)).std(0)

 
    train_perf_df = pd.DataFrame()
    test_perf_df = pd.DataFrame()
    train_perf_df['mean_pred_class0'] = train_observed_pred.mean.numpy()[0,]
    train_perf_df['mean_pred_class1'] = train_observed_pred.mean.numpy()[1,]
    train_perf_df['y'] = trainy
    train_perf_df['y_pred'] = train_pred_means.max(0)[1]
    train_perf_df['var_pred_class0']=train_observed_pred.variance.numpy()[0,]
    train_perf_df['var_pred_class1']=train_observed_pred.variance.numpy()[1,]
    train_perf_df['pred_prob_class0'] = train_probabilities.numpy()[0,]
    train_perf_df['pred_prob_class1'] = train_probabilities.numpy()[1,]
    train_perf_df['pred_prob_std_class0'] = train_prob_stds.numpy()[0,]
    train_perf_df['pred_prob_std_class1'] = train_prob_stds.numpy()[1,]
    
    train_perf_df['model'] = f'{root_name}_GP'
    train_perf_df['subset'] = 'train' 
    
    
    test_perf_df['mean_pred_class0'] = test_observed_pred.mean.numpy()[0,]
    test_perf_df['mean_pred_class1'] = test_observed_pred.mean.numpy()[1,]
    test_perf_df['y'] = testy
    test_perf_df['y_pred'] = test_pred_means.max(0)[1]
    test_perf_df['var_pred_class0']=test_observed_pred.variance.numpy()[0,]
    test_perf_df['var_pred_class1']=test_observed_pred.variance.numpy()[1,]
    test_perf_df['pred_prob_class0'] = test_probabilities.numpy()[0,]
    test_perf_df['pred_prob_class1'] = test_probabilities.numpy()[1,]
    test_perf_df['pred_prob_std_class0'] =test_prob_stds.numpy()[0,]
    test_perf_df['pred_prob_std_class1'] = test_prob_stds.numpy()[1,]
    test_perf_df['model'] = f'{root_name}_GP'
    test_perf_df['subset'] = 'test' 
    for k, val in train_results.items(): 
        train_perf_df[k] = val
    for k, val in test_results.items():
        test_perf_df[k] = val
    return train_perf_df, test_perf_df


In [15]:
data_dir = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/'
nek_nums = [2,3,5,9]
samplings = ['scaled', 'UNDER', 'SMOTE', 'ADASYN'] 
features = ['moe', 'mfp']
NEK= 'NEK'
for i, n in enumerate(nek_nums):
    nek = str(n)
    print(f'NEK{nek}')
    nek_path= f'{data_dir}NEK{nek}/bind/'
 
    for k, feat in enumerate(features): 
        print()
        for j, samp in enumerate(samplings): 
        
            print(f'NEK{nek} {feat} {samp}')
            file_root = f'NEK{nek}_binding_{feat}_{samp}'
            # print(file_root)
            trainX, trainy, testX, testy = make_torch_tens(nek_path,file_root)
            # train_perf_df = save_results(trainX, trainy, file_root, 'train', n_iterations=300, n_samples=256)
            # test_perf_df = save_results(testX, testy, file_root, 'test', n_iterations=300, n_samples=100)
            train_perf_df, test_perf_df = save_results(trainX, trainy, testX, testy,file_root, n_iterations=300)
            train_perf_df.to_csv(f'{nek_path}{file_root}_train_GP.csv',index=False) 
            test_perf_df.to_csv(f'{nek_path}{file_root}_test_GP.csv',index=False) 
            print(f'{nek_path}{file_root}_train_GP.csv')
            print(f'{nek_path}{file_root}_test_GP.csv')
            print()
            
            
    print()
    
    if n == 2 or n == 9:

        nek_path= f'{data_dir}NEK{nek}/inhib/'
        for k, feat in enumerate(features): 
            print()
            for j, samp in enumerate(samplings): 
                file_root = f'NEK{nek}_inhibition_{feat}_{samp}'
                print(f'NEK{nek} {feat} {samp}')
                trainX, trainy, testX, testy = make_torch_tens(nek_path,file_root)
                train_perf_df, test_perf_df = save_results(trainX, trainy, testX, testy,file_root, n_iterations=300)
                train_perf_df.to_csv(f'{nek_path}{file_root}_train_GP.csv',index=False) 
                test_perf_df.to_csv(f'{nek_path}{file_root}_test_GP.csv',index=False) 
                print(f'{nek_path}{file_root}_train_GP.csv')
                print(f'{nek_path}{file_root}_test_GP.csv')
                print()
        
            
    print()

NEK2

moe scaled
train X: torch.Size([1125, 306]), train y: torch.Size([1125]), test X: torch.Size([283, 306]), test y: torch.Size([283])
Iter 1/300 - Loss: 7.106   lengthscale: 0.693   noise: 0.693
Iter 11/300 - Loss: 5.484   lengthscale: 1.037   noise: 0.809
Iter 21/300 - Loss: 4.490   lengthscale: 1.398   noise: 1.073
Iter 31/300 - Loss: 3.896   lengthscale: 1.910   noise: 1.331
Iter 41/300 - Loss: 3.464   lengthscale: 2.568   noise: 1.482
Iter 51/300 - Loss: 3.290   lengthscale: 3.195   noise: 1.511
Iter 61/300 - Loss: 3.229   lengthscale: 3.633   noise: 1.458
Iter 71/300 - Loss: 3.199   lengthscale: 3.927   noise: 1.362
Iter 81/300 - Loss: 3.174   lengthscale: 4.136   noise: 1.250
Iter 91/300 - Loss: 3.159   lengthscale: 4.295   noise: 1.133
Iter 101/300 - Loss: 3.143   lengthscale: 4.421   noise: 1.020
Iter 111/300 - Loss: 3.140   lengthscale: 4.530   noise: 0.917
Iter 121/300 - Loss: 3.126   lengthscale: 4.622   noise: 0.829
Iter 131/300 - Loss: 3.118   lengthscale: 4.700   nois

/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/gpytorch/models/exact_gp.py:284: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(
/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/gpytorch/models/exact_gp.py:284: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(


accuracy: 0.9600, precision: 0.0000, recall: 0.0000, specificity: 1.0000, cm: [[1080    0]
 [  45    0]]
accuracy: 0.9576, precision: 0.0000, recall: 0.0000, specificity: 1.0000, cm: [[271   0]
 [ 12   0]]
/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/NEK2/bind/NEK2_binding_moe_scaled_train_GP.csv
/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/NEK2/bind/NEK2_binding_moe_scaled_test_GP.csv

moe UNDER
train X: torch.Size([90, 306]), train y: torch.Size([90]), test X: torch.Size([283, 306]), test y: torch.Size([283])
Iter 1/300 - Loss: 7.107   lengthscale: 0.693   noise: 0.693


/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Iter 11/300 - Loss: 5.931   lengthscale: 0.887   noise: 1.297
Iter 21/300 - Loss: 5.439   lengthscale: 1.231   noise: 1.982
Iter 31/300 - Loss: 5.262   lengthscale: 1.918   noise: 2.574
Iter 41/300 - Loss: 5.199   lengthscale: 2.952   noise: 3.007
Iter 51/300 - Loss: 5.173   lengthscale: 4.160   noise: 3.297
Iter 61/300 - Loss: 5.154   lengthscale: 5.325   noise: 3.464
Iter 71/300 - Loss: 5.140   lengthscale: 6.303   noise: 3.524
Iter 81/300 - Loss: 5.133   lengthscale: 7.003   noise: 3.513
Iter 91/300 - Loss: 5.129   lengthscale: 7.395   noise: 3.463
Iter 101/300 - Loss: 5.125   lengthscale: 7.557   noise: 3.393
Iter 111/300 - Loss: 5.122   lengthscale: 7.603   noise: 3.306
Iter 121/300 - Loss: 5.119   lengthscale: 7.615   noise: 3.202
Iter 131/300 - Loss: 5.116   lengthscale: 7.636   noise: 3.083
Iter 141/300 - Loss: 5.114   lengthscale: 7.671   noise: 2.953
Iter 151/300 - Loss: 5.111   lengthscale: 7.708   noise: 2.816
Iter 161/300 - Loss: 5.109   lengthscale: 7.733   noise: 2.675
I

...

accuracy: 0.9911, precision: 0.9893, recall: 0.9929, specificity: 0.9893, cm: [[277   3]
 [  2 278]]
accuracy: 0.9500, precision: 0.8571, recall: 0.6667, specificity: 0.9859, cm: [[70  1]
 [ 3  6]]
/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/NEK9/inhib/NEK9_inhibition_mfp_SMOTE_train_GP.csv
/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/NEK9/inhib/NEK9_inhibition_mfp_SMOTE_test_GP.csv

mfp ADASYN
train X: torch.Size([555, 2048]), train y: torch.Size([555]), test X: torch.Size([80, 2048]), test y: torch.Size([80])
Iter 1/300 - Loss: 6.282   lengthscale: 0.693   noise: 0.693


/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(


Iter 11/300 - Loss: 4.992   lengthscale: 1.312   noise: 0.798
Iter 21/300 - Loss: 4.359   lengthscale: 2.129   noise: 0.917
Iter 31/300 - Loss: 3.855   lengthscale: 3.039   noise: 0.697
Iter 41/300 - Loss: 3.606   lengthscale: 3.806   noise: 0.346
Iter 51/300 - Loss: 3.485   lengthscale: 4.278   noise: 0.138
Iter 61/300 - Loss: 3.431   lengthscale: 4.542   noise: 0.062
Iter 71/300 - Loss: 3.409   lengthscale: 4.690   noise: 0.035
Iter 81/300 - Loss: 3.397   lengthscale: 4.775   noise: 0.025
Iter 91/300 - Loss: 3.391   lengthscale: 4.828   noise: 0.019
Iter 101/300 - Loss: 3.386   lengthscale: 4.868   noise: 0.016
Iter 111/300 - Loss: 3.383   lengthscale: 4.903   noise: 0.013
Iter 121/300 - Loss: 3.380   lengthscale: 4.938   noise: 0.012
Iter 131/300 - Loss: 3.378   lengthscale: 4.975   noise: 0.010
Iter 141/300 - Loss: 3.376   lengthscale: 5.013   noise: 0.009
Iter 151/300 - Loss: 3.374   lengthscale: 5.050   noise: 0.008
Iter 161/300 - Loss: 3.372   lengthscale: 5.085   noise: 0.008
I

/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/gpytorch/models/exact_gp.py:284: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(
/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/gpytorch/models/exact_gp.py:284: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(
/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/Users/jayceepang/msse/capstone/gpytorch-venv/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 

accuracy: 0.9946, precision: 0.9892, recall: 1.0000, specificity: 0.9893, cm: [[277   3]
 [  0 275]]
accuracy: 0.9500, precision: 0.8571, recall: 0.6667, specificity: 0.9859, cm: [[70  1]
 [ 3  6]]
/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/NEK9/inhib/NEK9_inhibition_mfp_ADASYN_train_GP.csv
/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/NEK9/inhib/NEK9_inhibition_mfp_ADASYN_test_GP.csv




# Adding more metrics to csv results files 

In [45]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score, matthews_corrcoef, balanced_accuracy_score, confusion_matrix
import sys
sys.path.append('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/')
from RF_atomver import *

In [46]:
def add_cm(filepath, filename): 
    # print(filepath)
    # print(filename)
    df = pd.read_csv(filepath+filename)
    
    true_labels = df['y'] 
    predictions = df['y_pred']
    cm = confusion_matrix(true_labels, predictions )
    cm_flattened = cm.flatten().tolist()
    df['cm'] = [cm_flattened]* len(df)
    df['prediction_type'] = df.apply(lambda x: prediction_type(x['y'], x['y_pred']), axis=1)
    return df
        

In [ ]:
data_dir = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/'
gp_result_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_results/'
# capstone/atom2024/atom2024/notebooks/NEK/GP/GP_results/NEK2_binding_mfp_ADASYN_test_GP.csv
nek_nums = [2,3,5,9]
samplings = ['scaled', 'UNDER', 'SMOTE', 'ADASYN'] 
features = ['moe', 'mfp']
NEK= 'NEK'
for i, n in enumerate(nek_nums):
    nek = str(n)
    print(f'NEK{nek}')
    nek_path= f'{data_dir}NEK{nek}/bind/'
    
    for k, feat in enumerate(features): 
        print()
        for j, samp in enumerate(samplings): 
        
            print(f'NEK{nek} {feat} {samp}')
            file_root = f'NEK{nek}_binding_{feat}_{samp}'
            train_file = f'{file_root}_train_GP.csv'
            test_file = f'{file_root}_test_GP.csv'

            train_df = add_cm(gp_result_path,train_file)
            test_df = add_cm(gp_result_path,test_file)
            train_df.to_csv(f'{gp_result_path}{file_root}_train_GP.csv',index=False) 
            test_df.to_csv(f'{gp_result_path}{file_root}_test_GP.csv',index=False) 
            
            print()
            
            
    print()
    
    if n == 2 or n == 9:

        nek_path= f'{data_dir}NEK{nek}/inhib/'
        for k, feat in enumerate(features): 
            print()
            for j, samp in enumerate(samplings): 
                file_root = f'NEK{nek}_inhibition_{feat}_{samp}'
                print(f'NEK{nek} {feat} {samp}')
                print(f'NEK{nek} {feat} {samp}')
                file_root = f'NEK{nek}_inhibition_{feat}_{samp}'
                train_file = f'{file_root}_train_GP.csv'
                test_file = f'{file_root}_test_GP.csv'
    
                train_df = add_cm(gp_result_path,train_file)
                test_df = add_cm(gp_result_path,test_file)
                train_df.to_csv(f'{gp_result_path}{file_root}_train_GP.csv',index=False) 
                test_df.to_csv(f'{gp_result_path}{file_root}_test_GP.csv',index=False) 
                print()
        
            
    print()

In [39]:

# source = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/'
# dest = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_results/'

# if not os.path.exists(source):
#     os.makedirs(dest)

# for root, dirs, files in os.walk(source):
#     for file in files:
#         if file.endswith('_GP.csv'):
#             source_file = os.path.join(root, file)
#             dest_file = os.path.join(dest, file)
#             shutil.move(source_file,dest_file)
#             print(f"Moved: {source_file} to {dest_file}")


